In [29]:
import pandas as pd

In [30]:
def established_profitable_companies(
    df,
    max_market_rank=500,
    top_roa_threshold_pc=0.25,
    top_roe_threshold_pc=0.25,
    bottom_debt_to_equity_ratio=0.25,
) -> pd.DataFrame:
    """Focus on Established, Profitable Companies

    **Criteria:**

    *   **Market Capitalization**: Preferably large-cap stocks.
    *   **Profitability Ratios**: Positive and stable profit margins, operating margins, and gross margins.
    *   **Return Ratios**: High return on assets (ROA) and return on equity (ROE).
    *   **Debt Levels**: Low debt-to-equity ratio.
    *   **Dividend History**: Regular and stable dividends.

    Args:
        df (_type_): _description_
        max_market_rank (int, optional): _description_. Defaults to 500.
        top_roa_threshold_pc (float, optional): _description_. Defaults to 0.25.
        top_roe_threshold_pc (float, optional): _description_. Defaults to 0.25.
        bottom_debt_to_equity_ratio (float, optional): _description_. Defaults to 0.25.

    Returns:
        pd.DataFrame: Recommnded stocks
    """
    # Market Capitalization: Preferably large-cap stocks (manual segmentation needed)

    df_large_cap = df.query(f"market_cap_rank < {max_market_rank}")

    # Profitability Ratios: Positive and stable profit margins, operating margins, and gross margins
    df_profitable = df_large_cap.query(
        "profit_margins > 0 and operating_margins > 0 and gross_margins > 0"
    )

    # Return Ratios: High return on assets (ROA) and return on equity (ROE)
    # Top top_roa_threshold_pc% ROA and Top top_roe_threshold_pc% ROE
    roa_threshold = df_profitable["return_on_assets"].quantile(1 - top_roa_threshold_pc)
    roe_threshold = df_profitable["return_on_equity"].quantile(1 - top_roe_threshold_pc)

    df_high_return = df_profitable.query(
        f"return_on_assets > {roa_threshold} and return_on_equity > {roe_threshold}"
    )

    # Debt Levels: Low debt-to-equity ratio
    # Bottom bottom_debt_to_equity_ratio % debt-to-equity
    debt_to_equity_threshold = df_high_return["debt_to_equity"].quantile(
        bottom_debt_to_equity_ratio
    )

    df_low_debt = df_high_return.query(f"debt_to_equity < {debt_to_equity_threshold}")

    # Dividend History: Regular and stable dividends
    df_dividends = df_low_debt.dropna(
        subset=["ex_dividend_date", "last_dividend_value"]
    )

    # Final filtered dataframe
    df_filtered = df_dividends

    return df_filtered[
        [
            "symbol",
            "name_of_company",
            "day_high",
            "market_cap",
            "profit_margins",
            "operating_margins",
            "gross_margins",
            "return_on_assets",
            "return_on_equity",
            "debt_to_equity",
            "ex_dividend_date",
            "last_dividend_value",
        ]
    ]


def strong_financial_health_and_linquidity(
    df,
    min_quick_ratio=1,
    min_current_ratio=1,
    min_cash_to_debt_qn=0.8,
    min_free_cashflow=0,
    min_operating_cashflow=0,
):
    # Filtering criteria for strong financial health and liquidity

    # Quick Ratio: Greater than 1
    df_quick_ratio = df.query(f"quick_ratio > {min_quick_ratio}")

    # Current Ratio: Greater than 1
    df_current_ratio = df_quick_ratio.query(f"current_ratio > {min_current_ratio}")

    # Total Cash and Cash Per Share: High values relative to total debt

    cash_to_debt_threshold = (
        df_current_ratio["total_cash"] / df_current_ratio["total_debt"]
    ).quantile(min_cash_to_debt_qn)

    df_cash = df_current_ratio[
        (df_current_ratio["total_cash"] / df_current_ratio["total_debt"])
        > cash_to_debt_threshold
    ]

    # Free Cashflow and Operating Cashflow: Positive and growing
    df_cashflow = df_cash.query(
        f"free_cashflow > {min_free_cashflow} and operating_cashflow > {min_operating_cashflow}"
    )

    # Assuming 'previous_free_cashflow' and 'previous_operating_cashflow' columns exist for growth calculation
    # Uncomment the following lines if historical data is available for comparison
    # df_cashflow_growth = df_cashflow[
    #     (df_cashflow['free_cashflow'] > df_cashflow['previous_free_cashflow']) &
    #     (df_cashflow['operating_cashflow'] > df_cashflow['previous_operating_cashflow'])
    # ]

    # Display the filtered stocks
    return df_cashflow[
        [
            "symbol",
            "name_of_company",
            "day_high",
            "quick_ratio",
            "current_ratio",
            "total_cash",
            "total_cash_per_share",
            "total_debt",
            "free_cashflow",
            "operating_cashflow",
            "recommendation_key",
        ]
    ].sort_values(by=["day_high"])

In [31]:
df = pd.read_csv("../data/base/all_stocks.csv")

In [32]:
established_profitable_companies(df, max_market_rank=1000)

,symbol,name_of_company,day_high,market_cap,profit_margins,operating_margins,gross_margins,return_on_assets,return_on_equity,debt_to_equity,ex_dividend_date,last_dividend_value
52,MPSLTD,MPS Limited,2113.45,2.505398e+10,0.21780,0.24829,0.55367,0.14546,0.26886,0.987,1.721002e+09,45.0
72,NUCLEUS,Nucleus Software Exports Limited,1438.80,2.658642e+10,0.23183,0.25892,0.40018,0.12942,0.27606,0.590,1.720397e+09,12.5
126,SUPREMEIND,Supreme Industries Limited,6109.95,5.370479e+11,0.10555,0.14152,0.31824,0.12869,0.22494,1.078,1.718928e+09,22.0
165,CUMMINSIND,Cummins India Limited,4149.90,8.021614e+11,0.19117,0.25702,0.35880,0.14718,0.27817,1.920,1.721261e+09,20.0
192,SKFINDIA,SKF India Limited,6460.00,2.211310e+11,0.12074,0.16161,0.39208,0.11769,0.21966,0.438,1.720051e+09,130.0
273,TRITURBINE,Triveni Turbine Limited,646.80,1.412963e+11,0.16273,0.18457,0.50470,0.12445,0.31288,0.325,1.708042e+09,1.0
348,GANESHHOUC,Ganesh Housing Corporation Limited,1049.00,6.075251e+10,0.51648,0.53130,0.74293,0.24177,0.33775,1.680,1.724890e+09,11.0
398,CDSL,Central Depository Services (India) Limited,2348.00,1.699238e+11,0.46192,0.62260,1.00000,0.21505,0.30355,0.090,1.721088e+09,3.0
511,ESABINDIA,Esab India Limited,6171.90,6.572380e+10,0.13108,0.17611,0.40159,0.26876,0.57557,1.493,1.722470e+09,30.0
592,BLS,BLS International Services Limited,381.00,1.092429e+11,0.18664,0.17694,0.30055,0.15356,0.28579,2.122,1.708646e+09,0.5


In [33]:
strong_financial_health_and_linquidity(pd.read_csv("../data/base/all_stocks.csv"))

,symbol,name_of_company,day_high,quick_ratio,current_ratio,total_cash,total_cash_per_share,total_debt,free_cashflow,operating_cashflow,recommendation_key
709,SOMATEX,Soma Textiles & Industries Limited,34.74,1.716,1.831,3.503000e+08,10.629,16200000.0,1.391570e+08,2.593000e+08,none
1920,ACEINTEG,Ace Integrated Solutions Limited,34.87,7.054,12.270,3.230000e+07,3.841,2000000.0,8.262500e+06,3.700000e+06,none
1067,DIGIDRIVE,Digidrive Distributors Limited,36.85,3.743,4.748,1.810000e+08,4.684,900000.0,6.610000e+07,7.630000e+07,none
152,NIRAJ,Niraj Cement Structurals Limited,53.49,1.118,2.382,2.294390e+08,5.719,7242000.0,6.297372e+08,2.691250e+08,none
1374,DEN,Den Networks Limited,55.80,7.408,7.644,2.930769e+10,61.536,255110000.0,2.108471e+09,8.396700e+08,none
...,...,...,...,...,...,...,...,...,...,...,...
1832,VOLTAMP,Voltamp Transformers Limited,13468.90,2.107,3.709,6.060010e+08,59.899,32143000.0,1.600190e+09,2.162775e+09,buy
1613,ABBOTINDIA,Abbott India Limited,27997.15,1.827,2.417,2.134490e+10,1004.466,832300032.0,1.020105e+10,1.212810e+10,none
1864,BOSCHLTD,Bosch Limited,35680.00,1.094,1.948,3.389800e+10,1149.286,393000000.0,7.402625e+09,1.252800e+10,none
1424,3MINDIA,3M India Limited,39449.95,1.945,2.519,1.357843e+10,1205.344,181778000.0,5.623520e+09,6.430349e+09,none
